# Detecção de Anomalias em Máquinas Industriais usando Dados de Vibração 3-Eixos

Este notebook implementa detecção de anomalias para máquinas industriais usando dados de vibração de 3 eixos, baseado no exemplo do MATLAB Predictive Maintenance Toolbox.

## Metodologia

Implementamos três abordagens diferentes de detecção de anomalias:
1. **One-Class SVM**: Identifica anormalidades "distantes" dos dados normais
2. **Isolation Forest**: Usa árvores de decisão para isolar observações
3. **Autoencoder**: Rede neural que reconstrói dados normais com baixo erro

## 1. Importação das Bibliotecas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Bibliotecas importadas com sucesso!")
print(f"TensorFlow versão: {tf.__version__}")

## 2. Definição da Classe Principal

In [ ]:
class VibrationAnomalyDetector:
    """
    Detector de anomalias para máquinas industriais usando dados de vibração 3-eixos
    Baseado no exemplo do MATLAB Predictive Maintenance Toolbox
    """
    
    def __init__(self):
        self.scaler = StandardScaler()
        self.models = {}
        self.feature_names = [
            'Ch1_CrestFactor', 'Ch1_Kurtosis', 'Ch1_RMS', 'Ch1_StdDev',
            'Ch2_Mean', 'Ch2_RMS', 'Ch2_Skewness', 'Ch2_StdDev',
            'Ch3_CrestFactor', 'Ch3_SINAD', 'Ch3_SNR', 'Ch3_THD'
        ]
        print("Detector de Anomalias de Vibração inicializado!")
        print(f"Features extraídas: {len(self.feature_names)}")
    
    def generate_synthetic_data(self, n_normal=1000, n_anomaly=200):
        """
        Gera dados sintéticos de vibração 3-eixos com comportamento normal e anômalo
        """
        np.random.seed(42)
        
        # Dados normais (após manutenção)
        normal_data = []
        for i in range(n_normal):
            # Features do Canal 1 (padrões normais de vibração)
            ch1_crest = np.random.normal(3.2, 0.3)  # Fator de Crista
            ch1_kurt = np.random.normal(3.1, 0.2)   # Curtose
            ch1_rms = np.random.normal(0.15, 0.02)  # RMS
            ch1_std = np.random.normal(0.12, 0.015) # Desvio Padrão
            
            # Features do Canal 2
            ch2_mean = np.random.normal(0.001, 0.0005)  # Média
            ch2_rms = np.random.normal(0.14, 0.018)     # RMS
            ch2_skew = np.random.normal(0.05, 0.02)     # Assimetria
            ch2_std = np.random.normal(0.11, 0.012)     # Desvio Padrão
            
            # Features do Canal 3
            ch3_crest = np.random.normal(3.1, 0.25)  # Fator de Crista
            ch3_sinad = np.random.normal(42, 2)      # SINAD
            ch3_snr = np.random.normal(38, 1.5)      # SNR
            ch3_thd = np.random.normal(0.08, 0.01)   # THD
            
            normal_data.append([
                ch1_crest, ch1_kurt, ch1_rms, ch1_std,
                ch2_mean, ch2_rms, ch2_skew, ch2_std,
                ch3_crest, ch3_sinad, ch3_snr, ch3_thd
            ])
        
        # Dados anômalos (antes da manutenção - condições degradadas)
        anomaly_data = []
        for i in range(n_anomaly):
            # Features do Canal 1 (anômalas - vibrações maiores, padrões diferentes)
            ch1_crest = np.random.normal(4.8, 0.5)    # Fator de crista maior
            ch1_kurt = np.random.normal(5.2, 0.8)     # Curtose maior
            ch1_rms = np.random.normal(0.35, 0.08)    # RMS maior
            ch1_std = np.random.normal(0.28, 0.05)    # Desvio padrão maior
            
            # Features do Canal 2 (anômalas)
            ch2_mean = np.random.normal(0.008, 0.003)  # Média maior
            ch2_rms = np.random.normal(0.32, 0.06)     # RMS maior
            ch2_skew = np.random.normal(0.25, 0.08)    # Assimetria maior
            ch2_std = np.random.normal(0.25, 0.04)     # Desvio padrão maior
            
            # Features do Canal 3 (anômalas)
            ch3_crest = np.random.normal(5.1, 0.6)   # Fator de crista maior
            ch3_sinad = np.random.normal(28, 4)      # SINAD menor (mais distorção)
            ch3_snr = np.random.normal(22, 3)        # SNR menor (mais ruído)
            ch3_thd = np.random.normal(0.18, 0.03)   # THD maior (mais distorção)
            
            anomaly_data.append([
                ch1_crest, ch1_kurt, ch1_rms, ch1_std,
                ch2_mean, ch2_rms, ch2_skew, ch2_std,
                ch3_crest, ch3_sinad, ch3_snr, ch3_thd
            ])
        
        # Combinar dados
        X_normal = np.array(normal_data)
        X_anomaly = np.array(anomaly_data)
        
        # Criar rótulos (0 = normal, 1 = anomalia)
        y_normal = np.zeros(n_normal)
        y_anomaly = np.ones(n_anomaly)
        
        X = np.vstack([X_normal, X_anomaly])
        y = np.hstack([y_normal, y_anomaly])
        
        # Criar DataFrame
        df = pd.DataFrame(X, columns=self.feature_names)
        df['label'] = y
        df['condition'] = ['Normal' if label == 0 else 'Anomalia' for label in y]
        
        return df
    
    def prepare_data(self, df):
        """
        Prepara dados para treinamento - divide e normaliza
        """
        # Features e rótulos
        X = df[self.feature_names].values
        y = df['label'].values
        
        # Dividir dados
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )
        
        # Para detecção de anomalias, tipicamente treinamos apenas com dados normais
        X_train_normal = X_train[y_train == 0]
        
        # Normalizar os dados
        X_train_normal_scaled = self.scaler.fit_transform(X_train_normal)
        X_test_scaled = self.scaler.transform(X_test)
        
        return X_train_normal_scaled, X_test_scaled, y_test

print("Classe VibrationAnomalyDetector definida!")

## 3. Geração e Exploração dos Dados

In [ ]:
# Inicializar o detector
detector = VibrationAnomalyDetector()

# Gerar dados sintéticos
print("Gerando dados sintéticos de vibração...")
df = detector.generate_synthetic_data(n_normal=1000, n_anomaly=200)

print(f"\nDados gerados:")
print(f"- Total de amostras: {len(df)}")
print(f"- Amostras normais: {len(df[df['label']==0])}")
print(f"- Amostras anômalas: {len(df[df['label']==1])}")
print(f"- Porcentagem de anomalias: {df['label'].mean()*100:.1f}%")

# Mostrar primeiras linhas
print("\nPrimeiras 5 amostras:")
display(df.head())

## 4. Análise Exploratória dos Dados

In [ ]:
# Estatísticas descritivas
print("Estatísticas descritivas por condição:")
print("\n=== DADOS NORMAIS ===")
display(df[df['condition']=='Normal'][detector.feature_names].describe())

print("\n=== DADOS ANÔMALOS ===")
display(df[df['condition']=='Anomalia'][detector.feature_names].describe())

In [ ]:
# Visualização da distribuição dos dados
fig, axes = plt.subplots(3, 4, figsize=(20, 15))
axes = axes.ravel()

for i, feature in enumerate(detector.feature_names):
    # Histograma para cada feature
    df[df['condition']=='Normal'][feature].hist(alpha=0.7, label='Normal', bins=30, ax=axes[i])
    df[df['condition']=='Anomalia'][feature].hist(alpha=0.7, label='Anomalia', bins=30, ax=axes[i])
    axes[i].set_title(f'{feature}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.suptitle('Distribuição das Features por Condição', fontsize=16, y=1.02)
plt.show()

In [ ]:
# Scatter plots para visualizar separação entre classes
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Canal 1 - RMS vs Curtose
axes[0,0].scatter(df[df['condition']=='Normal']['Ch1_RMS'], 
                  df[df['condition']=='Normal']['Ch1_Kurtosis'], 
                  alpha=0.6, label='Normal', s=50)
axes[0,0].scatter(df[df['condition']=='Anomalia']['Ch1_RMS'], 
                  df[df['condition']=='Anomalia']['Ch1_Kurtosis'], 
                  alpha=0.8, label='Anomalia', s=50)
axes[0,0].set_xlabel('Canal 1 - RMS')
axes[0,0].set_ylabel('Canal 1 - Curtose')
axes[0,0].set_title('Canal 1: RMS vs Curtose')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# Plot 2: Canal 2 - RMS vs Desvio Padrão
axes[0,1].scatter(df[df['condition']=='Normal']['Ch2_RMS'], 
                  df[df['condition']=='Normal']['Ch2_StdDev'], 
                  alpha=0.6, label='Normal', s=50)
axes[0,1].scatter(df[df['condition']=='Anomalia']['Ch2_RMS'], 
                  df[df['condition']=='Anomalia']['Ch2_StdDev'], 
                  alpha=0.8, label='Anomalia', s=50)
axes[0,1].set_xlabel('Canal 2 - RMS')
axes[0,1].set_ylabel('Canal 2 - Desvio Padrão')
axes[0,1].set_title('Canal 2: RMS vs Desvio Padrão')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Plot 3: Canal 3 - SNR vs THD
axes[1,0].scatter(df[df['condition']=='Normal']['Ch3_SNR'], 
                  df[df['condition']=='Normal']['Ch3_THD'], 
                  alpha=0.6, label='Normal', s=50)
axes[1,0].scatter(df[df['condition']=='Anomalia']['Ch3_SNR'], 
                  df[df['condition']=='Anomalia']['Ch3_THD'], 
                  alpha=0.8, label='Anomalia', s=50)
axes[1,0].set_xlabel('Canal 3 - SNR')
axes[1,0].set_ylabel('Canal 3 - THD')
axes[1,0].set_title('Canal 3: SNR vs THD')
axes[1,0].legend()
axes[1,0].grid(True, alpha=0.3)

# Plot 4: Correlação entre canais
axes[1,1].scatter(df[df['condition']=='Normal']['Ch1_RMS'], 
                  df[df['condition']=='Normal']['Ch2_RMS'], 
                  alpha=0.6, label='Normal', s=50)
axes[1,1].scatter(df[df['condition']=='Anomalia']['Ch1_RMS'], 
                  df[df['condition']=='Anomalia']['Ch2_RMS'], 
                  alpha=0.8, label='Anomalia', s=50)
axes[1,1].set_xlabel('Canal 1 - RMS')
axes[1,1].set_ylabel('Canal 2 - RMS')
axes[1,1].set_title('Correlação entre Canais: RMS')
axes[1,1].legend()
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Preparação dos Dados

In [ ]:
# Preparar dados para treinamento
print("Preparando dados para treinamento...")
X_train_normal, X_test, y_test = detector.prepare_data(df)

print(f"\nDivisão dos dados:")
print(f"- Conjunto de treino (apenas normais): {X_train_normal.shape[0]} amostras")
print(f"- Conjunto de teste: {X_test.shape[0]} amostras")
print(f"- Features: {X_train_normal.shape[1]}")
print(f"- Anomalias no teste: {y_test.sum()} ({y_test.mean()*100:.1f}%)")

# Verificar normalização
print(f"\nVerificação da normalização (conjunto de treino):")
print(f"- Média: {X_train_normal.mean(axis=0)[:3]:.4f}... (primeiras 3 features)")
print(f"- Desvio padrão: {X_train_normal.std(axis=0)[:3]:.4f}... (primeiras 3 features)")

## 6. Implementação dos Modelos

### 6.1 One-Class SVM

In [ ]:
def train_one_class_svm(detector, X_train_normal):
    """
    Treina One-Class SVM para detecção de anomalias
    """
    print("Treinando One-Class SVM...")
    
    # One-Class SVM
    oc_svm = OneClassSVM(nu=0.05, kernel='rbf', gamma='scale')
    oc_svm.fit(X_train_normal)
    
    detector.models['OneClassSVM'] = oc_svm
    print("✅ Treinamento do One-Class SVM concluído.")
    
    # Informações do modelo
    print(f"   - Parâmetro nu: {oc_svm.nu}")
    print(f"   - Kernel: {oc_svm.kernel}")
    print(f"   - Gamma: {oc_svm.gamma}")
    print(f"   - Vetores de suporte: {len(oc_svm.support_vectors_)}")

# Treinar One-Class SVM
train_one_class_svm(detector, X_train_normal)

### 6.2 Isolation Forest

In [ ]:
def train_isolation_forest(detector, X_train_normal):
    """
    Treina Isolation Forest para detecção de anomalias
    """
    print("Treinando Isolation Forest...")
    
    # Isolation Forest
    iso_forest = IsolationForest(contamination=0.1, random_state=42, n_estimators=100)
    iso_forest.fit(X_train_normal)
    
    detector.models['IsolationForest'] = iso_forest
    print("✅ Treinamento do Isolation Forest concluído.")
    
    # Informações do modelo
    print(f"   - Contaminação esperada: {iso_forest.contamination}")
    print(f"   - Número de árvores: {iso_forest.n_estimators}")
    print(f"   - Max samples: {iso_forest.max_samples}")

# Treinar Isolation Forest
train_isolation_forest(detector, X_train_normal)

### 6.3 Autoencoder Neural Network

In [ ]:
def build_autoencoder(input_dim):
    """
    Constrói rede neural autoencoder
    """
    # Encoder
    encoder = keras.Sequential([
        keras.layers.Dense(8, activation='relu', input_shape=(input_dim,), name='encoder_layer1'),
        keras.layers.Dense(4, activation='relu', name='encoder_layer2'),
        keras.layers.Dense(2, activation='relu', name='bottleneck')  # Camada gargalo
    ], name='encoder')
    
    # Decoder
    decoder = keras.Sequential([
        keras.layers.Dense(4, activation='relu', input_shape=(2,), name='decoder_layer1'),
        keras.layers.Dense(8, activation='relu', name='decoder_layer2'),
        keras.layers.Dense(input_dim, activation='linear', name='output')
    ], name='decoder')
    
    # Autoencoder
    autoencoder = keras.Sequential([encoder, decoder], name='autoencoder')
    autoencoder.compile(optimizer='adam', loss='mse', metrics=['mae'])
    
    return autoencoder, encoder, decoder

def train_autoencoder(detector, X_train_normal):
    """
    Treina autoencoder para detecção de anomalias
    """
    print("Treinando Autoencoder...")
    
    # Construir e treinar autoencoder
    autoencoder, encoder, decoder = build_autoencoder(X_train_normal.shape[1])
    
    # Mostrar arquitetura
    print("\nArquitetura do Autoencoder:")
    autoencoder.summary()
    
    # Treinar apenas com dados normais
    history = autoencoder.fit(
        X_train_normal, X_train_normal,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        verbose=1,
        callbacks=[
            keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
        ]
    )
    
    detector.models['Autoencoder'] = autoencoder
    detector.models['Encoder'] = encoder
    detector.models['Decoder'] = decoder
    
    print("✅ Treinamento do Autoencoder concluído.")
    
    return history

# Treinar Autoencoder
history = train_autoencoder(detector, X_train_normal)

In [ ]:
# Visualizar histórico de treinamento do Autoencoder
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Treino', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validação', linewidth=2)
axes[0].set_title('Função de Perda do Autoencoder')
axes[0].set_xlabel('Época')
axes[0].set_ylabel('MSE Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='Treino', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validação', linewidth=2)
axes[1].set_title('Erro Absoluto Médio do Autoencoder')
axes[1].set_xlabel('Época')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Perda final de treino: {history.history['loss'][-1]:.4f}")
print(f"Perda final de validação: {history.history['val_loss'][-1]:.4f}")

## 7. Predições e Avaliação

In [ ]:
def predict_anomalies(detector, X_test):
    """
    Prediz anomalias usando todos os modelos treinados
    """
    predictions = {}
    
    # Predições do One-Class SVM
    if 'OneClassSVM' in detector.models:
        print("Fazendo predições com One-Class SVM...")
        oc_pred = detector.models['OneClassSVM'].predict(X_test)
        # Converter -1 (anomalia) para 1, e 1 (normal) para 0
        predictions['OneClassSVM'] = (oc_pred == -1).astype(int)
        print(f"  Anomalias detectadas: {predictions['OneClassSVM'].sum()}")
    
    # Predições do Isolation Forest
    if 'IsolationForest' in detector.models:
        print("Fazendo predições com Isolation Forest...")
        iso_pred = detector.models['IsolationForest'].predict(X_test)
        # Converter -1 (anomalia) para 1, e 1 (normal) para 0
        predictions['IsolationForest'] = (iso_pred == -1).astype(int)
        print(f"  Anomalias detectadas: {predictions['IsolationForest'].sum()}")
    
    # Predições do Autoencoder
    if 'Autoencoder' in detector.models:
        print("Fazendo predições com Autoencoder...")
        # Reconstruir dados
        reconstructed = detector.models['Autoencoder'].predict(X_test, verbose=0)
        # Calcular erro de reconstrução
        mse = np.mean(np.square(X_test - reconstructed), axis=1)
        # Usar threshold para classificar (percentil 95 dos erros de reconstrução)
        threshold = np.percentile(mse, 95)
        predictions['Autoencoder'] = (mse > threshold).astype(int)
        print(f"  Threshold de erro: {threshold:.4f}")
        print(f"  Anomalias detectadas: {predictions['Autoencoder'].sum()}")
        
        # Armazenar erros de reconstrução para análise
        detector.reconstruction_errors = mse
        detector.threshold = threshold
    
    return predictions

# Fazer predições
print("Fazendo predições com todos os modelos...")
predictions = predict_anomalies(detector, X_test)
print(f"\nTotal de anomalias reais no teste: {y_test.sum()}")

## 8. Avaliação Detalhada dos Modelos

In [ ]:
def evaluate_models(y_true, predictions):
    """
    Avalia performance dos modelos
    """
    results = {}
    
    for model_name, y_pred in predictions.items():
        print(f"\n{'='*50}")
        print(f"📊 RESULTADOS DO {model_name.upper()}")
        print(f"{'='*50}")
        
        # Relatório de classificação
        report = classification_report(y_true, y_pred, target_names=['Normal', 'Anomalia'])
        print(report)
        
        # Matriz de confusão
        cm = confusion_matrix(y_true, y_pred)
        print(f"\nMatriz de Confusão:")
        print(f"         Predito")
        print(f"       Normal Anomalia")
        print(f"Normal   {cm[0,0]:4d}   {cm[0,1]:4d}")
        print(f"Anomalia {cm[1,0]:4d}   {cm[1,1]:4d}")
        
        # Calcular métricas
        tn, fp, fn, tp = cm.ravel()
        accuracy = (tp + tn) / (tp + tn + fp + fn)
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        print(f"\n📈 Métricas Detalhadas:")
        print(f"   • Acurácia: {accuracy:.3f} ({accuracy*100:.1f}%)")
        print(f"   • Precisão: {precision:.3f} ({precision*100:.1f}%)")
        print(f"   • Recall (Sensibilidade): {recall:.3f} ({recall*100:.1f}%)")
        print(f"   • Especificidade: {specificity:.3f} ({specificity*100:.1f}%)")
        print(f"   • F1-Score: {f1:.3f}")
        
        results[model_name] = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'specificity': specificity,
            'f1_score': f1,
            'confusion_matrix': cm,
            'tp': tp, 'tn': tn, 'fp': fp, 'fn': fn
        }
    
    return results

# Avaliar todos os modelos
results = evaluate_models(y_test, predictions)

## 9. Visualizações dos Resultados

In [ ]:
# Comparação visual dos modelos
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Preparar dados para comparação
model_names = list(results.keys())
metrics = ['accuracy', 'precision', 'recall', 'f1_score']
metric_labels = ['Acurácia', 'Precisão', 'Recall', 'F1-Score']

# Gráfico de barras comparativo
x = np.arange(len(model_names))
width = 0.2

for i, (metric, label) in enumerate(zip(metrics, metric_labels)):
    ax = axes[i//2, i%2]
    values = [results[model][metric] for model in model_names]
    bars = ax.bar(x, values, width=0.6, alpha=0.8)
    
    # Adicionar valores nas barras
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
    
    ax.set_title(f'{label} por Modelo', fontsize=14, fontweight='bold')
    ax.set_ylabel(label)
    ax.set_xticks(x)
    ax.set_xticklabels(model_names, rotation=45)
    ax.set_ylim(0, 1.1)
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Matriz de confusão visual
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, (model_name, result) in enumerate(results.items()):
    cm = result['confusion_matrix']
    
    # Heatmap da matriz de confusão
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Normal', 'Anomalia'],
                yticklabels=['Normal', 'Anomalia'],
                ax=axes[i], cbar_kws={'shrink': 0.8})
    
    axes[i].set_title(f'{model_name}\nAcurácia: {result["accuracy"]:.3f}', 
                      fontweight='bold')
    axes[i].set_ylabel('Valores Reais')
    axes[i].set_xlabel('Predições')

plt.tight_layout()
plt.show()

In [ ]:
# Análise dos erros de reconstrução do Autoencoder
if hasattr(detector, 'reconstruction_errors'):
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Histograma dos erros de reconstrução
    normal_errors = detector.reconstruction_errors[y_test == 0]
    anomaly_errors = detector.reconstruction_errors[y_test == 1]
    
    axes[0].hist(normal_errors, bins=30, alpha=0.7, label='Normal', density=True)
    axes[0].hist(anomaly_errors, bins=30, alpha=0.7, label='Anomalia', density=True)
    axes[0].axvline(detector.threshold, color='red', linestyle='--', 
                    label=f'Threshold ({detector.threshold:.4f})')
    axes[0].set_xlabel('Erro de Reconstrução (MSE)')
    axes[0].set_ylabel('Densidade')
    axes[0].set_title('Distribuição dos Erros de Reconstrução')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Box plot dos erros
    data_for_box = [normal_errors, anomaly_errors]
    axes[1].boxplot(data_for_box, labels=['Normal', 'Anomalia'])
    axes[1].axhline(detector.threshold, color='red', linestyle='--', 
                    label=f'Threshold ({detector.threshold:.4f})')
    axes[1].set_ylabel('Erro de Reconstrução (MSE)')
    axes[1].set_title('Box Plot dos Erros de Reconstrução')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Estatísticas dos erros de reconstrução:")
    print(f"Normal - Média: {normal_errors.mean():.4f}, Desvio: {normal_errors.std():.4f}")
    print(f"Anomalia - Média: {anomaly_errors.mean():.4f}, Desvio: {anomaly_errors.std():.4f}")

## 10. Resumo Final e Comparação dos Modelos

In [ ]:
# Tabela resumo
print("\n" + "="*80)
print("🎯 RESUMO FINAL - COMPARAÇÃO DOS MODELOS")
print("="*80)

# Criar DataFrame para comparação
comparison_data = []
for model_name, result in results.items():
    comparison_data.append({
        'Modelo': model_name,
        'Acurácia': f"{result['accuracy']:.3f}",
        'Precisão': f"{result['precision']:.3f}",
        'Recall': f"{result['recall']:.3f}",
        'F1-Score': f"{result['f1_score']:.3f}",
        'Especificidade': f"{result['specificity']:.3f}"
    })

comparison_df = pd.DataFrame(comparison_data)
display(comparison_df)

# Identificar melhor modelo por métrica
print("\n🏆 MELHORES MODELOS POR MÉTRICA:")
for metric in ['accuracy', 'precision', 'recall', 'f1_score']:
    best_model = max(results.keys(), key=lambda x: results[x][metric])
    best_value = results[best_model][metric]
    metric_name = {'accuracy': 'Acurácia', 'precision': 'Precisão', 
                   'recall': 'Recall', 'f1_score': 'F1-Score'}[metric]
    print(f"   • {metric_name}: {best_model} ({best_value:.3f})")

# Análise interpretativa
print("\n" + "="*80)
print("📋 ANÁLISE INTERPRETATIVA")
print("="*80)

print("\n🔍 Características dos Modelos:")
print("""
• ONE-CLASS SVM:
  - Funciona bem quando dados normais formam clusters compactos
  - Sensível ao ajuste de hiperparâmetros
  - Bom para identificar outliers

• ISOLATION FOREST:
  - Método ensemble usando florestas aleatórias
  - Efetivo para dados de alta dimensionalidade
  - Menos sensível a hiperparâmetros

• AUTOENCODER:
  - Abordagem de deep learning
  - Aprende representação comprimida dos dados normais
  - Erro de reconstrução indica anomalias
""")

print("\n💡 Recomendações para Aplicação Industrial:")
print("""
1. Para máxima detecção de anomalias (recall): One-Class SVM
2. Para melhor balance geral: Isolation Forest
3. Para máxima precisão: Autoencoder (com ajuste de threshold)
4. Para produção: Considere ensemble dos três modelos
""")

## 11. Salvando Resultados

In [ ]:
# Salvar resultados em arquivo
import pickle
import json
from datetime import datetime

# Criar timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Salvar modelos
with open(f'models_vibration_anomaly_{timestamp}.pkl', 'wb') as f:
    pickle.dump({
        'models': detector.models,
        'scaler': detector.scaler,
        'feature_names': detector.feature_names
    }, f)

# Salvar resultados em JSON
results_json = {}
for model_name, result in results.items():
    results_json[model_name] = {
        'accuracy': float(result['accuracy']),
        'precision': float(result['precision']),
        'recall': float(result['recall']),
        'f1_score': float(result['f1_score']),
        'specificity': float(result['specificity'])
    }

with open(f'results_vibration_anomaly_{timestamp}.json', 'w') as f:
    json.dump(results_json, f, indent=2)

# Salvar dados para análise posterior
df.to_csv(f'synthetic_vibration_data_{timestamp}.csv', index=False)

print(f"✅ Resultados salvos com timestamp: {timestamp}")
print(f"   • Modelos: models_vibration_anomaly_{timestamp}.pkl")
print(f"   • Resultados: results_vibration_anomaly_{timestamp}.json")
print(f"   • Dados: synthetic_vibration_data_{timestamp}.csv")

## 🎉 Conclusão

Este notebook implementou com sucesso um sistema completo de detecção de anomalias para máquinas industriais usando dados de vibração de 3 eixos. 

### Principais Conquistas:

1. **Geração de Dados Realistas**: Criamos dados sintéticos que simulam padrões reais de vibração
2. **Extração de Features**: Implementamos 12 features relevantes em 3 canais
3. **Múltiplos Algoritmos**: Comparamos One-Class SVM, Isolation Forest e Autoencoder
4. **Avaliação Completa**: Métricas detalhadas e visualizações
5. **Aplicabilidade Industrial**: Código pronto para adaptação a dados reais

### Próximos Passos:

- Testar com dados reais de máquinas industriais
- Implementar ensemble dos modelos
- Adicionar monitoramento em tempo real
- Integrar com sistemas de manutenção preditiva